
# AAI612: Deep Learning & its Applications

*Notebook 7.1: Text Preprocessing Using *Scikit-learn*

<a href="https://colab.research.google.com/github/harmanani/AAI612/blob/main/Week7/Notebook7.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The CountVectorizer provides a simple way to both tokenize a collection of text documents and build a vocabulary of known words, but also to encode new documents using that vocabulary. You can use it as follows:

* Create an instance of the `CountVectorizer` class.
* Call the `fit()` function in order to learn a vocabulary from one or more documents.
* Call the `transform()` function on one or more documents as needed to encode each as a vector.

An encoded vector is returned with a length of the entire vocabulary and an integer count for the number of times each word appeared in the document. Because these vectors will contain a lot of zeros, we call them sparse. Python provides an efficient way of handling sparse vectors in the scipy.sparse package. The vectors returned from a call to transform() will be sparse vectors, and you can transform them back to NumPy arrays to look and better understand what is going on by calling the toarray() function. Below is an example of using the CountVectorizer to tokenize, build a vocabulary, and then encode a document.

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
# list of text documents
corpus = ["The quick brown fox jumped over the lazy dog."]

### Create the transform

In [2]:
vectorizer = CountVectorizer()

### Tokenize and Build Vocabulary

In [3]:
vectorizer.fit(corpus)

CountVectorizer()

Access the vocabulary to see what exactly was tokenized.  Notice that all words were made lowercase by default and that the punctuation was ignored:

In [4]:
vectorizer.vocabulary_

{'the': 7,
 'quick': 6,
 'brown': 0,
 'fox': 2,
 'jumped': 3,
 'over': 5,
 'lazy': 4,
 'dog': 1}

### Encode the document:

In [5]:
vector = vectorizer.transform(corpus)

### Summarize encoded vector

In [6]:
print(vector.shape)
print(vector.toarray())

(1, 8)
[[1 1 1 1 1 1 1 2]]


### Encode other documents

The same vectorizer can be used on documents that contain words not included in the vocabulary. These words are ignored and no count is given in the resulting vector. For example, below is an example of using the vectorizer above to encode a document with one word in the vocab and one word that is not:

In [7]:
# encode another document
text2 = ["the puppy"]
vector = vectorizer.transform(text2)
print(vector.toarray())

[[0 0 0 0 0 0 0 1]]


## Word Frequencies with TF-IDF

Word counts are a good starting point, but are very basic. One issue with simple counts is that some words like the will appear many times and their large counts will not be very meaningful in the encoded vectors. An alternative is to calculate word frequencies, and by far the most popular method is called TF-IDF. This is an acronym that stands for Term Frequency - Inverse Document Frequency which are the components of the resulting scores assigned to each word.
* **Term Frequency**: This summarizes how often a given word appears within a document.
* **Inverse Document Frequency**: This downscales words that appear a lot across documents.

Without going into the math, `TF-IDF` are word frequency scores that try to highlight words that are more interesting, e.g. frequent in a document but not across documents. The `TfidfVectorizer` will tokenize documents, learn the vocabulary and inverse document requency weightings, and allow you to encode new documents.

Alternately, if you already have a learned CountVectorizer, you can use it with a `TfidfTransformer` to just calculate the inverse document frequencies and start encoding documents. The same `create, fit, and transform` process is used as with the `CountVectorizer`.

### List of text documents

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = ["The quick brown fox jumped over the lazy dog.", "The dog.", "The fox"]

### Create the transform

In [9]:
vectorizer = TfidfVectorizer()

### Tokenize and build vocab

In [10]:
vectorizer.fit(corpus)

TfidfVectorizer()

### Summarize

In [11]:
print(vectorizer.vocabulary_)
print(vectorizer.idf_)

{'the': 7, 'quick': 6, 'brown': 0, 'fox': 2, 'jumped': 3, 'over': 5, 'lazy': 4, 'dog': 1}
[1.69314718 1.28768207 1.28768207 1.69314718 1.69314718 1.69314718
 1.69314718 1.        ]


### Encode document

In [12]:
vector = vectorizer.transform([corpus[0]])


### Summarize encoded vector

In [13]:
print(vector.shape)
print(vector.toarray())

(1, 8)
[[0.36388646 0.27674503 0.27674503 0.36388646 0.36388646 0.36388646
  0.36388646 0.42983441]]


A vocabulary of 8 words is learned from the documents and each word is assigned a unique integer index in the output vector. The inverse document frequencies are calculated for each word in the vocabulary, assigning the lowest score of 1.0 to the most frequently observed word: the at index 7. Finally, the first document is encoded as an 8-element sparse array and we can review the final scorings of each word with different values for the, fox, and dog from the other words in the vocabulary.  The scores are normalized to values between 0 and 1 and the encoded document vectors can
then be used directly with most machine learning algorithms.

In [14]:
texts = [
    "I love this phone. It's amazing!",
    "This is the worst laptop ever.",
    "The camera quality is great.",
    "Battery life is terrible.",
    "Absolutely fantastic product!",
    "Would not recommend this item.",
]
labels = [1, 0, 1, 0, 1, 0]

In [15]:
vectorizer = TfidfVectorizer(stop_words='english')

In [16]:
vectorizer

TfidfVectorizer(stop_words='english')

In [17]:
X_tfidf = vectorizer.fit_transform(texts)

In [19]:
X_tfidf.shape

(6, 16)

### the shape is 6 documment by 16 words that explains the shape

In [20]:
import pandas as pd
tfidf_df = pd.DataFrame(X_tfidf.toarray(), columns=vectorizer.get_feature_names_out())

In [21]:
tfidf_df

,absolutely,amazing,battery,camera,fantastic,great,item,laptop,life,love,phone,product,quality,recommend,terrible,worst
0,0.00000,0.57735,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.00000,0.57735,0.57735,0.00000,0.00000,0.000000,0.00000,0.000000
1,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,0.707107,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,0.00000,0.707107
2,0.00000,0.00000,0.00000,0.57735,0.00000,0.57735,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.57735,0.000000,0.00000,0.000000
3,0.00000,0.00000,0.57735,0.00000,0.00000,0.00000,0.000000,0.000000,0.57735,0.00000,0.00000,0.00000,0.00000,0.000000,0.57735,0.000000
4,0.57735,0.00000,0.00000,0.00000,0.57735,0.00000,0.000000,0.000000,0.00000,0.00000,0.00000,0.57735,0.00000,0.000000,0.00000,0.000000
5,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.707107,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.707107,0.00000,0.000000


TF-IDF(word, doc) = TF(word, doc) × IDF(word)
so for exampel amazing :

TF("amazing", Doc0) = 1 / 6 ≈ 0.333

it apears only once in the doc zero

IDF
n = 6  documents

df = 1 only document zero contains amazing

idf  = log(6/1+1) 1.09

tf-idf ( amazing) =  0.1


In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Optional: split data
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, labels, test_size=0.33, random_state=42)

# Train the model
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [24]:
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.5
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.50      1.00      0.67         1

    accuracy                           0.50         2
   macro avg       0.25      0.50      0.33         2
weighted avg       0.25      0.50      0.33         2



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


this model is not perfoming well since it has random prediction 0.5 we could try different models later but for now we focus on the text vectorization